In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')


# Linear Regression

In [ ]:
r = 0.6
x_demo = np.random.normal(0, 1, 10000)
z_demo = np.random.normal(0, 1, 10000)
y_demo = r*x_demo + np.sqrt(1 - r**2)*z_demo

In [ ]:
def trial_line():
    plots.figure(figsize=(7,7))
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)
    plots.scatter(x_demo, y_demo, s=10)
    #plots.plot([-4, 4], [-4*0.6,4*0.6], color='g', lw=2)
    plots.plot([-4,4],[-4,4], color='r', lw=2)
    #plots.plot([1.5,1.5], [-4,4], color='k', lw=2)
    plots.xlabel('x in standard units')
    plots.ylabel('y in standard units');

In [ ]:
def trial_with_vertical():
    trial_line()
    plots.plot([1.5,1.5], [-4,4], color='k', lw=2)

In [ ]:
def both_with_vertical():
    trial_line()
    plots.plot([1.5,1.5], [-4,4], color='k', lw=2)
    plots.plot([-4, 4], [-4*0.6,4*0.6], color='g', lw=2)

In [ ]:
def regression_line(r):
    x = np.random.normal(0, 1, 10000)
    z = np.random.normal(0, 1, 10000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.figure(figsize=(7, 7))
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)
    plots.scatter(x, y, s=10)
    plots.plot([-4, 4], [-4*r,4*r], color='g', lw=2)
    if r >= 0:
        plots.plot([-4,4],[-4,4], lw=2, color='r')
    else:
        plots.plot([-4,4], [4,-4], lw=2, color='r')

# Linear Regression

**Please run all cells before this cell, including the previous example cells and the import cell at the top of the notebook.**



In [ ]:
trial_line()

**Question:** Should we use this red line as our prediction line?

**Question:** Is this line the center for the y-values for each x-value?

To answer this lets look at x-2. What is the cetner of y-values there and what would our prediction for y be?
<br> The next line in python will show this

In [ ]:
trial_with_vertical()

**Question**: How should we change our line?
    <details>
<summary>Solution</summary>
  Let's flatten the line. The next line of code will do this
</details>

In [ ]:
both_with_vertical()

**Question:** What is the slope of the new line?

<details>
<summary>Solution</summary>
  Something between 0 and 1. 
    
</details>

**Question**: What exactly is the slope of this line?
<details>
<summary>Solution</summary>
  It is the correlation between these two variables? 
    
  Looking at the graph, we can see that for *n*-standard units of x, we should pick less than *n*-standard units of y 
    
  Pearson's and Galton's observation is that the slope of the prediction line is equal to the correlation
    
</details>

### Examples

Let's try different *r* values and see what happens in the figure

In [ ]:
r = 0.6
regression_line(r)

(back to slides)

# Equation of Regression Line

In [ ]:
def standard_units(x):
    return (x - np.average(x))/np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.average(x_su * y_su)

def slope(t, x, y):
    r = correlation(t, x, y)
    return r * np.std(t.column(y))/np.std(t.column(x))

def intercept(t, x, y):
    a = slope(t, x, y)
    return np.average(t.column(y)) - a*np.average(t.column(x))

In [ ]:
galton = Table.read_table('galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight'))
heights

In [ ]:
galton_slope = slope(heights, 'MidParent', 'Child')
galton_intercept = intercept(heights, 'MidParent', 'Child')
galton_slope, galton_intercept

**Question:** For a given midParent height of 68.3 inches, what will the predicted children's height be?


<details>
<summary>Solution</summary>
  galton_slope * 68.3 + galton_intercept
</details>



In [ ]:
# answer in python here

**Question**: Implement the function fitted_values below:
        
<details>
<summary>Solution</summary>
      a = slope(t, x, y)
    b = intercept(t, x, y)
    return a * t.column(x) + b
</details>


In [ ]:
def fitted_values(t, x, y):
    a = ...
    b = ...
    return ...

Now lets use this function to make predictions and plot the regression prediction in a scatter plot

In [ ]:
regression_predictions = fitted_values(heights, 'MidParent', 'Child')
heights = heights.with_column(
    'Regression Prediction', regression_predictions
)
heights.scatter('MidParent')

(back to slides)

# Root Mean Squared Error

In [ ]:
little_women = Table().read_table('https://www.inferentialthinking.com/data/little_women.csv')
little_women

In [ ]:
little_women.scatter(1,0)

**Question:** Do we think there is a linear association here?

**Question:** Let's compute the correlation

In [ ]:
correlation(little_women, 1, 0)

Now lets predict the number of characters based on the number of periods in a chapter

In [ ]:
pred_characters = ...
pred_characters 

In [ ]:
little_women_fitted = little_women.with_columns("fitted", pred_characters)
little_women_fitted.hist() # What does this histogram mean? 
# Then lets uncomment the next line to look at the scatter of the predictions
#little_women_fitted.scatter(1)

## Squared Error

This function will draw a line with a specified slope and intercept and will draw red lines showing the errors

In [ ]:
sample = [[131, 14431], [231, 20558], [392, 40935], [157, 23524]]
def lw_errors(slope, intercept):
    print("Slope:      ", np.round(slope), 'characters per period')
    print("Intercept:  ", np.round(intercept), 'characters')
    little_women.scatter('Periods', 'Characters')
    xlims = np.array([50, 450])
    plots.plot(xlims, slope * xlims + intercept, lw=2)
    for x,y in sample:
        plots.plot([x, x], [y, slope * x + intercept], color='r', lw=2)

In [ ]:
lw_errors(50, 1000)

Let's try it with more slopes and intercepts

In [ ]:
lw_errors(70, 1000)

Lets make a line that goes through part of the scatter diagram

Now let's compute the root mean square error for the differnt lines we just considered

In [ ]:
def lw_rmse(slope, intercept):
    lw_errors(slope, intercept)
    x = little_women.column('Periods')
    y = little_women.column('Characters')
    prediction = slope * x + intercept
    mse = np.mean((y - prediction) ** 2)
    print("Root mean squared error:", round(mse ** 0.5, 2))

In [ ]:
lw_rmse(, )

In [ ]:
lw_rmse(, )

In [ ]:
lw_rmse(, )

In [ ]:
lw_rmse(, )

Let's comput the rmse for the regression line 

<details>
<summary>Solution</summary>
  lw_rmse(slope(little_women, 1, 0), intercept(little_women, 1, 0))
</details>

In [ ]:
lw_rmse( , )

The regression line is the line that minimzies the root mean squared error.

(back to slides)
# Least Squares

In [ ]:
x = np.arange(1, 3, 0.1)
y = (x-2)**2 + 3
Table().with_columns('x', x, 'y', y).plot('x')

In [ ]:
def f(x):
    return ((x-2)**2) + 3

What x-value gives us the smalles y-value?

In [ ]:
minimize(f)

In [ ]:
x = np.arange(-1.5, 1.5, 0.05)
y2 = 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 
Table().with_columns('x', x, 'y', y2).plot('x')

In [ ]:
def complicated_function(x):
    return 2 * np.sin(x*np.pi) + x ** 3 + x ** 4

What x-value gives us the smalles y-value?

In [ ]:
minimize(complicated_function)

(back to slides)
# Minimizing MSE


In [ ]:
def lw_mse(any_slope, any_intercept):
    x = little_women.column('Periods')
    y = little_women.column('Characters')
    estimate = any_slope*x + any_intercept
    return (np.mean((y - estimate) ** 2)) 

In [ ]:
lw_rmse(1500, 20000), lw_mse(1500, 20000)

In [ ]:
minimize(lw_mse)

(back to slides)
# Residuals 

Let's make a new function called residuals

In [ ]:
def residuals(t, x, y):
    """ Returns residual for each prediction, 
        i.e. the difference between the true y and predicted y"""
    predictions = ...
    return t.column(y) - predictions

In [ ]:
residuals(little_women, "Periods", "Characters")

Let's add residuals to a table and plot the table
    <details>
<summary>Solution</summary>
  little_women_fitted.with_columns('residuals', residuals(little_women, "Periods", "Characters")).scatter('Periods')
</details>

In [ ]:
  little_women_fitted.with_columns('residuals', residuals(little_women, "Periods", "Characters")).scatter('Periods')

**Question:** Why are the residuals on the bottom?
        <details>
<summary>Solution</summary>
  Becuase residuals show the difference between the prediction and the true value
</details>

Let's plot the residuals and the predictions seperately

In [ ]:
def plot_residuals(t, x, y):
    with_residuals = t.with_columns(
        "Fitted", fitted_values(t, x, y),
        "Residual", residuals(t, x, y)/ 1000 # I did this division just for this example
    )
    with_residuals.select(x, y, 'Fitted').scatter(0)
    with_residuals.scatter(x, 'Residual')

In [ ]:
plot_residuals(little_women, "Periods", "Characters")

We see the residuals clustered around 0. This makes sense because the data is correlated

(back to demo)
# Nonlinear Regression

We will look at two examples


In [ ]:
shotput = Table.read_table('shotput.csv')
shotput

In [ ]:
correlation(shotput, "Weight Lifted", "Shot Put Distance")

In [ ]:
shotput.scatter('Weight Lifted')

**Question**: Is the association linear?

In [ ]:
plot_residuals(shotput, "Weight Lifted", "Shot Put Distance")

Let's look at another example

In [ ]:
height_weight = Table.read_table('us_women.csv')
height_weight

**Question**: Is the association linear?

In [ ]:
correlation(height_weight, 'height', 'ave weight')

In [ ]:
plot_residuals(height_weight, 'height', "ave weight")

(back to slides) 
# A Measure of cluster

In [ ]:
def plot_fitted(t, x, y):
    tbl = t.select(x, y)
    tbl.with_columns('Fitted Value', fitted_values(t, x, y)).scatter(0)

In [ ]:
plot_fitted(height_weight, 'height', "ave weight")

In [ ]:
weight_pred_sd = np.std(fitted_values(height_weight, 'height', 'ave weight'))
weight_observed_sd = np.std(height_weight.column('ave weight'))
print(weight_pred_sd)
print(weight_observed_sd)

In [ ]:
weight_pred_sd / weight_observed_sd

In [ ]:
correlation(height_weight, 'height', 'ave weight')